# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f208cace130>
├── 'a' --> tensor([[ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108]])
└── 'x' --> <FastTreeValue 0x7f208cace970>
    └── 'c' --> tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                        [ 1.8341, -0.3364, -0.5752, -0.2783],
                        [ 0.9362, -0.8803, -0.7308,  1.1749]])

In [4]:
t.a

tensor([[ 2.1633,  0.7938,  1.3251],
        [ 2.1342,  0.5409, -0.3108]])

In [5]:
%timeit t.a

57.6 ns ± 0.081 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f208cace130>
├── 'a' --> tensor([[ 0.8492, -0.8751,  1.1343],
│                   [-0.8008, -0.9553,  0.8969]])
└── 'x' --> <FastTreeValue 0x7f208cace970>
    └── 'c' --> tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                        [ 1.8341, -0.3364, -0.5752, -0.2783],
                        [ 0.9362, -0.8803, -0.7308,  1.1749]])

In [7]:
%timeit t.a = new_value

57.4 ns ± 0.0357 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.1633,  0.7938,  1.3251],
               [ 2.1342,  0.5409, -0.3108]]),
    x: Batch(
           c: tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749]]),
       ),
)

In [10]:
b.a

tensor([[ 2.1633,  0.7938,  1.3251],
        [ 2.1342,  0.5409, -0.3108]])

In [11]:
%timeit b.a

56.6 ns ± 0.0304 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2873, -1.5631,  0.3232],
               [ 1.3276, -0.1846,  1.1624]]),
    x: Batch(
           c: tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.0931 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

856 ns ± 0.192 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f208ca55310>
├── 'a' --> tensor([[[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]],
│           
│                   [[ 2.1633,  0.7938,  1.3251],
│                    [ 2.1342,  0.5409, -0.3108]]])
└── 'x' --> <FastTreeValue 0x7f208ca55280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f205bfcbc10>
├── 'a' --> tensor([[ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108],
│                   [ 2.1633,  0.7938,  1.3251],
│                   [ 2.1342,  0.5409, -0.3108]])
└── 'x' --> <FastTreeValue 0x7f208cabadf0>
    └── 'c' --> tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                        [ 1.8341, -0.3364, -0.5752, -0.2783],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.9 µs ± 58.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0411,  0.6047, -0.2491, -2.0091],
                       [ 1.8341, -0.3364, -0.5752, -0.2783],
                       [ 0.9362, -0.8803, -0.7308,  1.1749]],
              
                      [[ 1.0411,  0.6047, -0.2491, -2.0091],
                       [ 1.8341, -0.3364, -0.5752, -0.2783],
                       [ 0.9362, -0.8803, -0.7308,  1.1749]],
              
                      [[ 1.0411,  0.6047, -0.2491, -2.0091],
                       [ 1.8341, -0.3364, -0.5752, -0.2783],
                       [ 0.9362, -0.8803, -0.7308,  1.1749]],
              
                      [[ 1.0411,  0.6047, -0.2491, -2.0091],
                       [ 1.8341, -0.3364, -0.5752, -0.2783],
                       [ 0.9362, -0.8803, -0.7308,  1.1749]],
              
                      [[ 1.0411,  0.6047, -0.2491, -2.0091],
                       [ 1.8341, -0.3364, -0.5752, -0.2783],
                       [ 0.9362, -0.8803, -0.7308,  1.1749]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749],
                      [ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749],
                      [ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749],
                      [ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749],
                      [ 1.0411,  0.6047, -0.2491, -2.0091],
                      [ 1.8341, -0.3364, -0.5752, -0.2783],
                      [ 0.9362, -0.8803, -0.7308,  1.1749],
                      [ 1.0411,  0.6047, -0.2491, -2.0091],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
